Scientific programming in Python (Winter 2025/26)

# Sheet 02: numpy

## Introduction

You can solve this sheet alone or in your group. Once done, upload your solution to your group's folder (under [`Groups`](https://studip.uni-osnabrueck.de/dispatch.php/course/files/index/6e9e531b6e84cadd07ac27fd03fb8493?cid=64105ba596df4fcc36ec266431aac621)).  Please do so before **October 28th, 2025**.  You should be able to explain your solution to your tutor in the feedback meeting.  You should solve at least 50% of the exercises on each sheet, to participate in the project work of this course, which will start after the new year break.

## Goal of this Sheet

This sheet introduces core NumPy concepts: arrays, dtypes, views vs copies, broadcasting, vectorization, indexing/masking, and basic linear algebra. 

Setup: you need to have `numpy` installed for this sheet. Run the following cell to check:

In [1]:
import numpy as np #kommentar

np.__version__

'2.3.3'

## Exercise 1: Arrays, dtypes, shapes, and views vs copies

Motivation: NumPy arrays are homogeneous, strided memory blocks with powerful operations. Understanding shape, dtype, and views is foundational for writing efficient scientific code.

### a) Create, reshape, and inspect

- Create a 1D array a containing the integers 0..11 (inclusive).
- Reshape it into a 3x4 array A and then into a 4x3 array B.
- Report shapes, dtype, itemsize, and total nbytes for A and B.

In [2]:
import numpy as np

a = np.arange(12)
A = a.reshape(3, 4)
B = a.reshape(4, 3)
print("A:", A.shape, A.dtype, A.itemsize, A.nbytes)
print("B:", B.shape, B.dtype, B.itemsize, B.nbytes)

assert A.shape == (3, 4), "A must be 3x4"
assert B.shape == (4, 3), "B must be 4x3"
assert A.size == 12 and B.size == 12, "A and B should contain 12 elements"

A: (3, 4) int64 8 96
B: (4, 3) int64 8 96


### b) Slicing and views

- Take the last two columns of A and assign to C using slicing (no copy).
- Modify C in-place (e.g., add 100) and verify that A changed accordingly.

In [3]:
C = A[:, 2:]
C += 100
print("A after modifying C:\n", A)

assert np.all(A[:, 2:] >= 100), "Last two columns of A should have been increased by at least 100"
assert C.base is A or getattr(C, 'base', None) is not None, "C should be a view, not a copy"

A after modifying C:
 [[  0   1 102 103]
 [  4   5 106 107]
 [  8   9 110 111]]


A changed because slices of an array are always views, so you just "view" the same chunk of memory from a different perspective. This saves a lot of memory, but it means also that the original array will be changed if you change the viewed slice as we did with C.

### c) ravel vs flatten

- Produce R = A.ravel() and F = A.flatten(). Modify R[0] and F[1].
- Check which changes reflect back in A and explain briefly.

In [7]:
R = A.ravel()
F = A.flatten()
R[0] = -999
F[1] = -888
print("A after modifications:\n", A)

assert A.ravel()[0] == -999, "Modifying R should reflect in A if R is a view"
assert F.base is None, "F should be a copy (flatten)"

A after modifications:
 [[-999    1  202  203]
 [   4    5  206  207]
 [   8    9  210  211]]


`flatten` always returns a copy, meaning it occupies a different chunk of memory than the array it operates one and any modifications done with this are not reflected by the original array. In contrast, `ravel` returns a contiguous view of the original array whenever possible, so any modifications of R should be reflected by A. 

### d) Write a function that returns a centered block view

Implement take_center_block(a, block_shape) that returns a view into the center of a 2D array a.

In [20]:
def take_center_block(a: np.ndarray, block_shape: tuple[int, int]) -> np.ndarray:
    """
    Return a view of the centered block of shape block_shape from 2D array a.
    """
    center_row_range = int((a.shape[0] - block_shape[0]) / 2)  # calculate how many rows have to be "cut off" from each side as the halved difference between the number of rows in the original array and the desired block shape 
    center_column_range = int((a.shape[1] - block_shape[1]) / 2)  # do the same for the columns
    center_block_a = a[center_row_range:-(center_row_range), center_column_range:-(center_column_range)]  # take out the center block with the desired dimensions
    return center_block_a
    # YOUR CODE HERE
    

# BEGIN TESTS
T = np.arange(7*9).reshape(7, 9)
blk = take_center_block(T, (3, 5))
assert blk.shape == (3, 5), "Block has wrong shape"
blk[:] = -1
assert np.all(T[2:5, 2:7] == -1), "Function should return a view into the center block"
X = np.arange(10*10).reshape(10, 10)
b = take_center_block(X, (4, 6))
assert np.shares_memory(b, X), "Must return a view, not a copy"
# END TESTS

## Exercise 2: Broadcasting and vectorization

Vectorization and broadcasting are core to efficient NumPy. Replace Python loops with array operations whenever possible.

### a) Column-wise z-score normalization with broadcasting.

Handle zero variance via eps.

In [15]:
import numpy as np

# Consistent RNG
rng = np.random.default_rng(42)

def zscore(X: np.ndarray, axis=0, eps=1e-12) -> np.ndarray:
    """
    Return z-scored array along axis. Zero-variance handled by eps.
    """
    mean = np.mean(X, axis)  # calculate the mean of each column
    std_dev = np.std(X, axis)  # calculate the standard deviation of each column
    
    if std_dev.any() < eps:  # prevent dividing by 0
        std_dev += eps
        
    z_scores = (X - mean) / std_dev  # column-wise z-score normalization 

    return z_scores

# BEGIN TESTS
X = rng.normal(size=(1000, 3))
Z = zscore(X, axis=0)
assert np.allclose(Z.mean(axis=0), 0, atol=5e-2)
assert np.allclose(Z.std(axis=0), 1, atol=5e-2)

Xc = np.ones((10, 5)) * 7.5
Zh = zscore(Xc, axis=0)
assert np.allclose(Zh, 0, atol=1e-8), "Constant columns should become ~0 after z-scoring with eps"
# END TESTS

AttributeError: 'NoneType' object has no attribute 'mean'

### b) Pairwise Euclidean distance matrix using broadcasting.

Assume two sets of $d$-dimensional vectors, $A$ containing $N$ vectors and $B$ containing $M$ vectors: compute an (N, M) matrix containing the pairwise Euclidean distances between pairs of vectors from $A$ and $B$.

In [12]:
import numpy as np
def pairwise_distances(A: np.ndarray, B: np.ndarray) -> np.ndarray:
    """
    Return an (N, M) matrix of Euclidean distances between rows of A and B.
    """

    
    # YOUR CODE HERE
    diff = B-A[:, np.newaxis, :]
    D = np.sqrt(np.sum(diff**2, axis=-1))
    return D

# BEGIN TESTS
A_sm = np.array([[0., 0.], [1., 0.], [0., 1.]])
B_sm = np.array([[0., 0.], [1., 1.]])
D = pairwise_distances(A_sm, B_sm)
expected = np.array([[0.0, np.sqrt(2)], [1.0, 1.0], [1.0, 1.0]])
assert np.allclose(D, expected), "Distances incorrect for the small example"
# END TESTS

In [86]:
from math import sqrt 
A_sm = np.array([[0., 0.], [1., 0.], [0., 1.]])
B_sm = np.array([[0., 0.], [1., 1.]])


type(A_sm[1])

#distances = sqrt((A_sm-B_sm)**2)



ValueError: operands could not be broadcast together with shapes (3,2) (2,2) 

### c) Performance comparison

Implement a simple loop baseline and verify equality with vectorized version.

In [16]:
def pairwise_distances_loop(A, B):
    """
    Return an (N, M) matrix of Euclidean distances between rows of A and B, computed using loops.
    """
    # YOUR CODE HERE
    for i in range(A.shape[0]):
        for j in range(B.shape[0]):
            if i == 0 and j == 0:
                D = np.empty((A.shape[0], B.shape[0]))
            D[i, j] = np.sqrt(np.sum((A[i] - B[j])**2))
    return D
# BEGIN TESTS
A_big = rng.normal(size=(80, 3))
B_big = rng.normal(size=(75, 3))
#Dv = pairwise_distances(A_big, B_big)
Dl = pairwise_distances_loop(A_big, B_big)
#assert np.allclose(Dv, Dl, atol=1e-10), "Vectorized and loop results must match"
# END TESTS

#%timeit pairwise_distances(A_big, B_big)
#%timeit pairwise_distances_loop(A_big, B_big)

244 μs ± 10.1 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
57 ms ± 814 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [98]:
from math import sqrt
A_big = rng.normal(size=(80, 3))
B_big = rng.normal(size=(75, 3))

distances = np.zeros((A_big.shape[0], B_big.shape[0]))
for i in A_big:
    for j in B_big:
        distances = sqrt((i-j)**2)

distances


TypeError: only length-1 arrays can be converted to Python scalars

## Exercise 3: Indexing, masking, and data cleaning

Real-world data contains outliers and missing values. Boolean masks and fancy indexing help you filter and clean efficiently.

### a) Simulate data with outliers

Create an array of shape (T=1000 datapoints, S=5 sensors), containing simulated sensor readings.  Sensor readings are normaly distributed, with variance 1 and with different means per sensor. Inject ~1% spikes by adding values between +10 and +20 at random positions.

In [ ]:
import numpy as np

# Consistent RNG
rng = np.random.default_rng(42)

T, S = 1000, 5
# means = ...
# readings = ...

# YOUR CODE HERE

# BEGIN TESTS
assert readings.shape == (1000, 5)
assert np.any(readings - base > 9.9), "Outliers not injected properly"
# We should see a noticeable right-skew in max vs median
assert np.nanmax(readings) - np.nanmedian(readings) > 8.0
# END TESTS

### 3b) Detect outliers via robust z-score using MAD

For each sensor (column), compute median and [Median Absolute Deviation](https://en.wikipedia.org/wiki/Median_absolute_deviation) (MAD, that is the median of the absolute deviations from the median), and then the *robust z-score* (rsz, sometimes also called "modified z-score")

$$rzs = 0.6745\cdot\frac{x - \operatorname{median}}{\operatorname{MAD}}$$

Flag $|rzs| > 3.5$ as outliers. Use eps to avoid division by zero.

In [ ]:
eps = 1e-12
# eps = ...
# median = ...
# mad = ...
# rzs = ...
# outliers = ...

# YOUR CODE HERE

# BEGIN TESTS
assert outliers.shape == readings.shape
assert outliers.sum() > 0, "Expect some outliers"
# END TESTS

# END QUESTION

### c) Replace outliers with NaN and compute per-sensor stats

Create a new array callede `cleaned` containing the original readings, with outliers replaced by NaN.cleaned[outliers] = np.nan.

Then compute per-sensor mean and std ignoring NaNs.

Hint: have a look at the documentation for [numpy.nanmean](https://numpy.org/doc/stable/reference/generated/numpy.nanmean.html).

In [ ]:
# cleaned = ...
# mean_per_sensor = ...
# std_per_sensor = ...

# YOUR CODE HERE

# BEGIN TESTS
assert cleaned.shape == readings.shape
assert np.all(np.isfinite(mean_per_sensor)), "Means should be finite"
assert mean_per_sensor.shape == (S,)
# Removing outliers should reduce max
assert np.nanmax(cleaned) <= np.nanmax(readings)
# END TESTS

### d) Impute missing values with per-sensor median

Compute per-sensor medians ignoring NaNs and fill NaNs in cleaned.

In [ ]:
# sensor_medians = ...
# imputed = ...

# YOUR CODE HERE

# BEGIN TESTS
assert not np.isnan(imputed).any(), "No NaNs should remain after imputation"
assert imputed.shape == (T, S)
# Imputed values in outlier positions should be close to column medians
bad = np.where(outliers)
assert np.allclose(imputed[bad], sensor_medians[0, bad[1]], atol=1e-12)
# END TESTS